In [32]:
dierctory_name = "pose_2011_spi"



In [24]:
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt

import copy
from scipy.stats import zscore

import os
import re

h = 0
w = 1


In [25]:
# 画像からサイズを取り出す
def get_image_size(img_name):
    img_ = cv2.imread(img_name)
    return np.delete(np.array(img_.shape), 2)

In [26]:
# targetの決定とkptの取り出し(reshape済)
def take_out_kpt(data, img_size):
    
    #print("人数:"+str(len(data["people"])))

    x = [] 
    y = []

    x_mean = []
    y_mean = []
    kpt_num = []

    for people in data['people']:
        kpt = np.array(people['pose_keypoints']).reshape((18, 3))

        #x,yに分ける
        for i in range(18):
            if kpt[i,2] != 0:
                x = np.append(x,kpt[i,0])
                y = np.append(y,kpt[i,1])

        #print("kpt_num:"+str(len(x)))

        #keypointの数、x,y座標の平均値（中央を原点とした、）
        kpt_num = np.append(kpt_num, len(x))
        x_mean = np.append(x_mean, np.mean(x - img_size[w] / 2))
        y_mean = np.append(y_mean, np.mean(y - img_size[h] / 2))

    
    #人の情報(x,y座標)を集約
    people_info = np.append(x_mean,y_mean).reshape(2,-1)
    #print("kpt_nums:"+str(kpt_num))
    #print("xms:"+str(x_mean))
    #print("yms:"+str(y_mean))
    #print("xy_argmins:"+str(np.argmin(np.absolute(people_info),axis=1)))
    
    
    #絶対値の最小のインデックスを導出
    xy_argmin = np.argmin(np.absolute(people_info),axis=1)
    #print("xy_argmin +kpt_num_argmax:"+str(np.append(xy_argmin, np.argmax(kpt_num))))

    #keypointの数、x,y座標の平均値を比較し、最も多い人のインデックスを導出
    target_index = np.argmax(np.bincount(np.append(xy_argmin, np.argmax(kpt_num))))
    #print(np.bincount(np.append(xy_argmin, np.argmax(kpt_num))))

    #targetの情報取り出し
    people_data = data["people"]
    target = people_data[target_index]

    target_kpt = np.array(target["pose_keypoints"]).reshape(18,3)

    return target_kpt

In [27]:
# 標準化
def conv_kpt_zscore(kpt):
    #kpt_zscore = []
    x = [] 
    y = []

    for i in range(18):
        if kpt[i,2] != 0:
            x = np.append(x,kpt[i,0])
            y = np.append(y,kpt[i,1])

    x_zscore = zscore(x)
    y_zscore = zscore(y)

    j = 0
    for i in range(18):
        if(kpt[i,2]) != 0:
            kpt[i,0] = copy.copy(x_zscore[j])
            kpt[i,1] = copy.copy(y_zscore[j])
            j += 1
    #print(kpt)


In [31]:
with open('image0019_keypoints.json', 'r') as f1:
    data = json.load(f1)

img_size = get_image_size("image0001_rendered.png")
kpt = take_out_kpt(data, img_size)
print(kpt)
conv_kpt_zscore(kpt)
print(kpt)

[[  2.30691000e+02   1.76059000e+02   7.56858000e-02]
 [  2.58974000e+02   1.69258000e+02   3.51334000e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  2.59954000e+02   1.76118000e+02   3.28530000e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  3.08730000e+02   1.90683000e+02   6.35355000e-01]
 [  2.92176000e+02   2.24846000e+02   8.68456000e-01]
 [  3.23371000e+02   2.65804000e+02   8.34199000e-01]
 [  3.27253000e+02   1.82897000e+02   5.96848000e-01]
 [  3.72134000e+02   2.15099000e+02   6.44217000e-01]
 [  4.18950000e+02   2.40462000e+02   6.60176000e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  2.31685000e+02   1.70208000e+02   4.07526000e-01]]
[[-1.23918853 -0.78715963  

In [30]:
def json_files_open(dierctory_name):
    
    # 画像からサイズを取り出す
    def get_image_size(img_name):
        img_ = cv2.imread(img_name)
        return np.delete(np.array(img_.shape), 2)

    # targetの決定とkptの取り出し(reshape済)
    def take_out_kpt(data, img_size):

        #print("人数:"+str(len(data["people"])))

        x = [] 
        y = []

        x_mean = []
        y_mean = []
        kpt_num = []

        for people in data['people']:
            kpt = np.array(people['pose_keypoints']).reshape((18, 3))

            #x,yに分ける
            for i in range(18):
                if kpt[i,2] != 0:
                    x = np.append(x,kpt[i,0])
                    y = np.append(y,kpt[i,1])

            #print("kpt_num:"+str(len(x)))

            #keypointの数、x,y座標の平均値（中央を原点とした、）
            kpt_num = np.append(kpt_num, len(x))
            x_mean = np.append(x_mean, np.mean(x - img_size[w] / 2))
            y_mean = np.append(y_mean, np.mean(y - img_size[h] / 2))

        #人の情報(x,y座標)を集約
        people_info = np.append(x_mean,y_mean).reshape(2,-1)
        #print("kpt_nums:"+str(kpt_num))
        #print("xms:"+str(x_mean))
        #print("yms:"+str(y_mean))
        #print("xy_argmins:"+str(np.argmin(np.absolute(people_info),axis=1)))


        #絶対値の最小のインデックスを導出
        xy_argmin = np.argmin(np.absolute(people_info),axis=1)
        #print("xy_argmin +kpt_num_argmax:"+str(np.append(xy_argmin, np.argmax(kpt_num))))

        #keypointの数、x,y座標の平均値を比較し、最も多い人のインデックスを導出
        target_index = np.argmax(np.bincount(np.append(xy_argmin, np.argmax(kpt_num))))
        #print(np.bincount(np.append(xy_argmin, np.argmax(kpt_num))))

        #targetの情報取り出し
        people_data = data["people"]
        target = people_data[target_index]
        target_kpt = np.array(target["pose_keypoints"]).reshape(18,3)

        return target_kpt

    
    # 標準化
    def conv_kpt_zscore(kpt):
        #kpt_zscore = []
        x = [] 
        y = []

        for i in range(18):
            if kpt[i,2] != 0:
                x = np.append(x,kpt[i,0])
                y = np.append(y,kpt[i,1])

        x_zscore = zscore(x)
        y_zscore = zscore(y)

        j = 0
        for i in range(18):
            if(kpt[i,2]) != 0:
                kpt[i,0] = copy.copy(x_zscore[j])
                kpt[i,1] = copy.copy(y_zscore[j])
                j += 1

    _json = re.compile("json")
    _png = re.compile("png")

    all_frame_kpt = []

    dierctory = os.listdir(dierctory_name)

    # ディレクトリの移動
    os.chdir(dierctory_name)

    # 画像サイズの取り出し
    for file in dierctory:
        if _png.search(file):
            try:
                img_size = get_image_size(file)
                print(file+"\t:get_image_size success")
                break
            except:
                print(file+"\t:get_image_size failed")

    # jsonファイルオープンとデータ取出し
    n = 0
    for file in dierctory:
        if _json.search(file):
            try:
                with open(file, 'r') as f2:
                    data = json.load(f2)

                #ここでデータ取出し、標準化する
                kpt = take_out_kpt(data, img_size)
                conv_kpt_zscore(kpt)
                all_frame_kpt = np.append(all_frame_kpt,kpt)

                print(file+"\t:success")
                n += 1

            except:
                print(file+"\t:failed")

    print(n)
    os.chdir("../")
    
    return all_frame_kpt


image0001_rendered.png	:get_image_size success
image0001_keypoints.json	:success
image0002_keypoints.json	:success
image0003_keypoints.json	:success
image0004_keypoints.json	:success
image0005_keypoints.json	:success
image0006_keypoints.json	:success
image0007_keypoints.json	:success
image0008_keypoints.json	:success
image0009_keypoints.json	:success
image0010_keypoints.json	:success
image0011_keypoints.json	:success
image0012_keypoints.json	:success
image0013_keypoints.json	:success
image0014_keypoints.json	:success
image0015_keypoints.json	:success
image0016_keypoints.json	:success
image0017_keypoints.json	:success
image0018_keypoints.json	:success
image0019_keypoints.json	:success
image0020_keypoints.json	:success
image0021_keypoints.json	:success
image0022_keypoints.json	:success
image0023_keypoints.json	:success
image0024_keypoints.json	:success
image0025_keypoints.json	:success
image0026_keypoints.json	:success
image0027_keypoints.json	:success
image0028_keypoints.json	:success
i